<a href="https://colab.research.google.com/github/smmurdock/Solve-an-Optimization-Problem/blob/main/d605_task_3_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D605 Task 3 - Identification of the Objective Function and Constraints

## A. Optimization Problem Program

> Using the mathematical representations that you developed in Task 2, write a program in either Python or R to solve the optimization problem computationally.

In [12]:
!pip install pulp

In [13]:
import pulp
import sys

In [14]:
# --- Data definitions as described from "Amazon Distribution.docx" ---
# Define the sets of locations
hubs = ['CVG', 'AFW']
focus_cities = ['Leipzig', 'Hyderabad', 'San Bernardino']
centers = [
    'Paris', 'Cologne', 'Hanover', 'Bangalore',
    'Coimbatore', 'Delhi', 'Mumbai', 'Cagliari',
    'Catania', 'Milan', 'Rome', 'Katowice',
    'Barcelona', 'Madrid', 'Castle Donington', 'London',
    'Mobile', 'Anchorage', 'Fairbanks', 'Phoenix',
    'Los Angeles', 'Ontario', 'Riverside', 'Sacramento',
    'San Francisco', 'Stockton', 'Denver', 'Hartford',
    'Miami', 'Lakeland', 'Tampa', 'Atlanta',
    'Honolulu', 'Kahului/Maui', 'Kona', 'Chicago',
    'Rockford', 'Fort Wayne', 'South Bend', 'Des Moines',
    'Wichita', 'New Orleans', 'Baltimore', 'Minneapolis',
    'Kansas City', 'St. Louis', 'Omaha', 'Manchester',
    'Albuquerque', 'New York', 'Charlotte', 'Toledo',
    'Wilmington', 'Portland', 'Allentown', 'Pittsburgh',
    'San Juan', 'Nashville', 'Austin', 'Dallas',
    'Houston', 'San Antonio', 'Richmond', 'Seattle/Tacoma',
    'Spokane'
]

# Define capacities for hubs and focus cities
hub_capacities = {
    'CVG': 95_650,
    'AFW': 44_350
}

focus_city_capacities = {
    'Leipzig': 85_000,
    'Hyderabad': 19_000,
    'San Bernardino': 36_000
}

# Define demand for each fulfillment center
center_demands = {
    'Paris': 6500, 'Cologne': 640, 'Hanover': 180, 'Bangalore': 9100,
    'Coimbatore': 570, 'Delhi': 19000, 'Mumbai': 14800, 'Cagliari': 90,
    'Catania': 185, 'Milan': 800, 'Rome': 1700, 'Katowice': 170,
    'Barcelona': 2800, 'Madrid': 3700, 'Castle Donington': 30, 'London': 6700,
    'Mobile': 190, 'Anchorage': 175, 'Fairbanks': 38, 'Phoenix': 2400,
    'Los Angeles': 7200, 'Ontario': 100, 'Riverside': 1200, 'Sacramento': 1100,
    'San Francisco': 1900, 'Stockton': 240, 'Denver': 1500, 'Hartford': 540,
    'Miami': 3400, 'Lakeland': 185, 'Tampa': 1600, 'Atlanta': 3000,
    'Honolulu': 500, 'Kahului/Maui': 16, 'Kona': 63, 'Chicago': 5100,
    'Rockford': 172, 'Fort Wayne': 200, 'South Bend': 173, 'Des Moines': 300,
    'Wichita': 290, 'New Orleans': 550, 'Baltimore': 1300, 'Minneapolis': 1700,
    'Kansas City': 975, 'St. Louis': 1200, 'Omaha': 480, 'Manchester': 100,
    'Albuquerque': 450, 'New York': 11200, 'Charlotte': 900, 'Toledo': 290,
    'Wilmington': 150, 'Portland': 1200, 'Allentown': 420, 'Pittsburgh': 1000,
    'San Juan': 1100, 'Nashville': 650, 'Austin': 975, 'Dallas': 3300,
    'Houston': 3300, 'San Antonio': 1100, 'Richmond': 600, 'Seattle/Tacoma': 2000,
    'Spokane': 260
}

# Define transportation costs
# Keys: tuples (source, destination)
# Value: float (distribution cost)
costs = {
    # Hub to Focus City
    ('CVG', 'Leipzig'): 1.5, ('CVG', 'San Bernardino'): 0.5,
    ('AFW', 'San Bernardino'): 0.5,
    ('Leipzig', 'Hyderabad'): 1.6,
    # Hub to Fulfillment Center
    ('CVG', 'Paris'): 1.6, ('CVG', 'Cologne'): 1.5, ('CVG', 'Hanover'): 1.5, ('CVG', 'Cagliari'): 1.5,
    ('CVG', 'Catania'): 1.5, ('CVG', 'Milan'): 1.5, ('CVG', 'Rome'): 1.5, ('CVG', 'Katowice'): 1.4,
    ('CVG', 'Barcelona'): 1.5, ('CVG', 'Madrid'): 1.6, ('CVG', 'Castle Donington'): 1.4, ('CVG', 'London'): 1.6,
    ('CVG', 'Mobile'): 0.5, ('CVG', 'Anchorage'): 1.3, ('CVG', 'Fairbanks'): 1.4, ('CVG', 'Phoenix'): 0.5,
    ('CVG', 'Los Angeles'): 0.5, ('CVG', 'Ontario'): 0.5, ('CVG', 'Riverside'): 0.5, ('CVG', 'Sacramento'): 0.5,
    ('CVG', 'San Francisco'): 0.5, ('CVG', 'Stockton'): 0.5, ('CVG', 'Denver'): 0.5, ('CVG', 'Hartford'): 0.5,
    ('CVG', 'Miami'): 0.5, ('CVG', 'Lakeland'): 0.5, ('CVG', 'Tampa'): 0.5, ('CVG', 'Atlanta'): 0.5,
    ('CVG', 'Chicago'): 0.5, ('CVG', 'Rockford'): 0.5, ('CVG', 'Fort Wayne'): 0.5, ('CVG', 'South Bend'): 0.5,
    ('CVG', 'Des Moines'): 0.5, ('CVG', 'Wichita'): 0.5, ('CVG', 'New Orleans'): 0.5, ('CVG', 'Baltimore'): 0.5,
    ('CVG', 'Minneapolis'): 0.5, ('CVG', 'Kansas City'): 0.5, ('CVG', 'St. Louis'): 0.5, ('CVG', 'Omaha'): 0.5,
    ('CVG', 'Manchester'): 0.5, ('CVG', 'Albuquerque'): 0.5, ('CVG', 'New York'): 0.5, ('CVG', 'Charlotte'): 0.5,
    ('CVG', 'Toledo'): 0.5, ('CVG', 'Wilmington'): 0.5, ('CVG', 'Portland'): 0.5, ('CVG', 'Allentown'): 0.5,
    ('CVG', 'Pittsburgh'): 0.5, ('CVG', 'San Juan'): 0.5, ('CVG', 'Nashville'): 0.5, ('CVG', 'Austin'): 0.5,
    ('CVG', 'Dallas'): 0.5, ('CVG', 'Houston'): 0.5, ('CVG', 'San Antonio'): 0.5, ('CVG', 'Richmond'): 0.5,
    ('CVG', 'Seattle/Tacoma'): 0.5, ('CVG', 'Spokane'): 0.5, ('AFW', 'Mobile'): 0.5, ('AFW', 'Anchorage'): 1.0,
    ('AFW', 'Fairbanks'): 1.0, ('AFW', 'Phoenix'): 0.5, ('AFW', 'Los Angeles'): 0.5, ('AFW', 'Ontario'): 0.5,
    ('AFW', 'Riverside'): 0.5, ('AFW', 'Sacramento'): 0.5, ('AFW', 'San Francisco'): 0.5, ('AFW', 'Stockton'): 0.5,
    ('AFW', 'Denver'): 0.5, ('AFW', 'Hartford'): 0.5, ('AFW', 'Miami'): 0.5, ('AFW', 'Lakeland'): 0.5,
    ('AFW', 'Tampa'): 0.5, ('AFW', 'Atlanta'): 0.5, ('AFW', 'Honolulu'): 0.5, ('AFW', 'Kahului/Maui'): 0.5,
    ('AFW', 'Kona'): 0.5, ('AFW', 'Chicago'): 0.5, ('AFW', 'Rockford'): 0.5, ('AFW', 'Fort Wayne'): 0.5,
    ('AFW', 'South Bend'): 0.5, ('AFW', 'Des Moines'): 0.5, ('AFW', 'Wichita'): 0.5, ('AFW', 'New Orleans'): 0.5,
    ('AFW', 'Baltimore'): 0.5, ('AFW', 'Minneapolis'): 0.5, ('AFW', 'Kansas City'): 0.5, ('AFW', 'St. Louis'): 0.5,
    ('AFW', 'Omaha'): 0.5, ('AFW', 'Manchester'): 0.5, ('AFW', 'Albuquerque'): 0.5, ('AFW', 'New York'): 0.5,
    ('AFW', 'Charlotte'): 0.5, ('AFW', 'Toledo'): 0.5, ('AFW', 'Wilmington'): 0.5, ('AFW', 'Portland'): 0.5,
    ('AFW', 'Allentown'): 0.5, ('AFW', 'Pittsburgh'): 0.5, ('AFW', 'San Juan'): 0.5, ('AFW', 'Nashville'): 0.5,
    ('AFW', 'Austin'): 0.25, ('AFW', 'Houston'): 0.25, ('AFW', 'San Antonio'): 0.25, ('AFW', 'Richmond'): 0.5,
    ('AFW', 'Seattle/Tacoma'): 0.5, ('AFW', 'Spokane'): 0.5,
    # Focus City to Fulfillment Center
    ('Leipzig', 'Paris'): 0.5, ('Leipzig', 'Cologne'): 0.5, ('Leipzig', 'Hanover'): 0.5, ('Leipzig', 'Bangalore'): 1.5,
    ('Leipzig', 'Coimbatore'): 1.5, ('Leipzig', 'Delhi'): 1.5, ('Leipzig', 'Mumbai'): 1.5, ('Leipzig', 'Cagliari'): 0.5,
    ('Leipzig', 'Catania'): 0.5, ('Leipzig', 'Milan'): 0.5, ('Leipzig', 'Rome'): 0.5, ('Leipzig', 'Katowice'): 0.5,
    ('Leipzig', 'Barcelona'): 0.5, ('Leipzig', 'Madrid'): 0.5, ('Leipzig', 'Castle Donington'): 0.5, ('Leipzig', 'London'): 0.75,
    ('Leipzig', 'Hartford'): 1.5, ('Leipzig', 'Baltimore'): 1.5, ('Leipzig', 'Manchester'): 1.5, ('Leipzig', 'New York'): 1.6,
    ('Leipzig', 'Allentown'): 1.5, ('Hyderabad', 'Paris'): 1.1, ('Hyderabad', 'Cologne'): 1.0, ('Hyderabad', 'Hanover'): 1.0,
    ('Hyderabad', 'Bangalore'): 0.5, ('Hyderabad', 'Coimbatore'): 0.5, ('Hyderabad', 'Delhi'): 0.5, ('Hyderabad', 'Mumbai'): 0.5,
    ('Hyderabad', 'Cagliari'): 1.0, ('Hyderabad', 'Catania'): 1.0, ('Hyderabad', 'Milan'): 1.0, ('Hyderabad', 'Rome'): 1.1,
    ('Hyderabad', 'Katowice'): 1.0, ('Hyderabad', 'Barcelona'): 1.0, ('Hyderabad', 'Madrid'): 1.1, ('Hyderabad', 'London'): 1.1,
    ('San Bernardino', 'Mobile'): 0.5, ('San Bernardino', 'Anchorage'): 0.7, ('San Bernardino', 'Fairbanks'): 0.7, ('San Bernardino', 'Phoenix'): 0.5,
    ('San Bernardino', 'Sacramento'): 0.5, ('San Bernardino', 'San Francisco'): 0.5, ('San Bernardino', 'Stockton'): 0.5, ('San Bernardino', 'Denver'): 0.5,
    ('San Bernardino', 'Hartford'): 0.5, ('San Bernardino', 'Miami'): 0.7, ('San Bernardino', 'Lakeland'): 0.7, ('San Bernardino', 'Tampa'): 0.7,
    ('San Bernardino', 'Atlanta'): 0.6, ('San Bernardino', 'Honolulu'): 0.5, ('San Bernardino', 'Kahului/Maui'): 0.5, ('San Bernardino', 'Kona'): 0.5,
    ('San Bernardino', 'Chicago'): 0.5, ('San Bernardino', 'Rockford'): 0.5, ('San Bernardino', 'Fort Wayne'): 0.5, ('San Bernardino', 'South Bend'): 0.5,
    ('San Bernardino', 'Des Moines'): 0.5, ('San Bernardino', 'Wichita'): 0.5, ('San Bernardino', 'New Orleans'): 0.5, ('San Bernardino', 'Baltimore'): 0.7,
    ('San Bernardino', 'Minneapolis'): 0.5, ('San Bernardino', 'Kansas City'): 0.5, ('San Bernardino', 'St. Louis'): 0.5, ('San Bernardino', 'Omaha'): 0.5,
    ('San Bernardino', 'Manchester'): 0.7, ('San Bernardino', 'Albuquerque'): 0.5, ('San Bernardino', 'New York'): 0.7, ('San Bernardino', 'Charlotte'): 0.7,
    ('San Bernardino', 'Toledo'): 0.5, ('San Bernardino', 'Wilmington'): 0.7, ('San Bernardino', 'Portland'): 0.5, ('San Bernardino', 'Allentown'): 0.7,
    ('San Bernardino', 'Pittsburgh'): 0.6, ('San Bernardino', 'San Juan'): 1.0, ('San Bernardino', 'Nashville'): 0.5, ('San Bernardino', 'Austin'): 0.5,
    ('San Bernardino', 'Dallas'): 0.5, ('San Bernardino', 'Houston'): 0.5, ('San Bernardino', 'San Antonio'): 0.5, ('San Bernardino', 'Richmond'): 0.7,
    ('San Bernardino', 'Seattle/Tacoma'): 0.5, ('San Bernardino', 'Spokane'): 0.5
}

The code cell above defines all the data needed for each hub, focus city, and fulfillment center, including capacities and distribution costs.

In [15]:
# --- Problem initialization ---
# Create the linear programming problem
# Resource: (PuLP Documentation - A Transportation Problem)
prob = pulp.LpProblem("Amazon_Air_Cargo_Optimization", pulp.LpMinimize)

The code above initializes the Linear Programming solver model from the PuLP library, set to find the optimal minimum.

In [16]:
# --- Decision Variables ---
# x_ij, y_ik, z_jk

# x_ij: hub i to focus city j
# Create tuples of valid routes for x
routes_x = [(h, fc) for h in hubs for fc in focus_cities if (h, fc) in costs]
x = pulp.LpVariable.dicts("Shipment_Hub_to_Focus_City", routes_x, lowBound=0, cat='Continuous')

# y_ik: hub i to fulfillment center k
# Create tuples of valid routes for y
routes_y = [(h, c) for h in hubs for c in centers if (h, c) in costs]
y = pulp.LpVariable.dicts("Shipment_Hub_to_Fulfillment_Center", routes_y, lowBound=0, cat='Continuous')

# z_jk: focus city j to fulfillment center k
# Create tuples of valid routes for z
routes_z = [(fc, c) for fc in focus_cities for c in centers if (fc, c) in costs]
z = pulp.LpVariable.dicts("Shipment_Focus_to_Fulfillment_Center", routes_z, lowBound=0, cat='Continuous')

The code above programmatically defines the constraints.

In [17]:
# --- Objective function ---
prob += (
    pulp.lpSum([x[route] * costs[route] for route in routes_x]) +
    pulp.lpSum([y[route] * costs[route] for route in routes_y]) +
    pulp.lpSum([z[route] * costs[route] for route in routes_z]),
    "Total_Transportation_Cost",
)

The code above programmatically adds the objective function to the initialized problem solver.

In [18]:
# --- Constraints ---
# 1. Hub Capacity Constraints
for h in hubs:
    prob += (
        pulp.lpSum([x[(h, fc)] for fc in focus_cities if (h, fc) in routes_x]) +
        pulp.lpSum([y[(h, c)] for c in centers if (h, c) in routes_y])
        <= hub_capacities[h],
        f"Hub_Capacity_{h}",
    )

# 2. Focus City Capacity Constraints (Inbound)
for fc in focus_cities:
    prob += (
        pulp.lpSum([x[(h, fc)] for h in hubs if (h, fc) in routes_x])
        <= focus_city_capacities[fc],
        f"Focus_City_Capacity_Inbound_{fc}",
    )

# 3. Flow Balance at Focus Cities (Outbound must equal inbound)
for fc in focus_cities:
    prob += (
        pulp.lpSum([z[(fc, c)] for c in centers if (fc, c) in routes_z])
        == pulp.lpSum([x[(h, fc)] for h in hubs if (h, fc) in routes_x]),
        f"Flow_Balance_{fc}",
    )

# 4. Fulfillment Center Demand Constraints
for c in centers:
    prob += (
        pulp.lpSum([y[(h, c)] for h in hubs if (h, c) in routes_y]) +
        pulp.lpSum([z[(fc, c)] for fc in focus_cities if (fc, c) in routes_z])
        == center_demands[c],
        f"Demand_Satisfaction_{c.replace(' ', '_').replace('/', '_')}",
    )

The code above adds the identified constraints to the initialized problem solver.

### A1. Demonstrate Solver Solution

> Demonstrate that the solver provided a solution to the optimization problem.

In [19]:
# --- Solve the Problem ---
# The problem is solved using PuLP's default solver (CBC)
prob.solve()

# Display the Results
print("=" * 80)
print(f"Optimization Solution")
print("=" * 80)
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Cost: ${pulp.value(prob.objective):,.2f}")

Optimization Solution
Status: Optimal
Total Cost: $199,476.25


The code above uses attributes generated from the problem solver to demonstrate that we have returned an optimal status and that the total cost of shipments equals **$199,476.25**.

## B. Output Analysis

> Analyze the output of your model to demonstrate that the solution satisfies the requirements of the problem.

In [23]:
# --- Solve the Problem ---
# The problem is solved using PuLP's default solver (CBC)
prob.solve()

# Display the Results
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Cost: ${pulp.value(prob.objective):,.2f}")

Status: Optimal
Total Cost: $199,476.25


In [24]:
# --- Display the Plan ---

print("\nOptimal Shipping Plan")
print("-" * 80)

for i, j in sorted(x.keys()):
    if x[i, j].varValue > 0:
        print(f"Hub {i} to Focus City {j}: {x[i, j].varValue:,.0f} tons")
for i, k in sorted(y.keys()):
    if y[i, k].varValue > 0:
        print(f"Hub {i} to Center {k}: {y[i, k].varValue:,.0f} tons")
for j, k in sorted(z.keys()):
    if z[j, k].varValue > 0:
        print(f"Focus City {j} to Center {k}: {z[j, k].varValue:,.0f} tons")


Optimal Shipping Plan
--------------------------------------------------------------------------------
Hub CVG to Focus City Leipzig: 43,470 tons
Hub AFW to Center Albuquerque: 450 tons
Hub AFW to Center Allentown: 420 tons
Hub AFW to Center Anchorage: 175 tons
Hub AFW to Center Atlanta: 3,000 tons
Hub AFW to Center Austin: 975 tons
Hub AFW to Center Charlotte: 900 tons
Hub AFW to Center Chicago: 5,100 tons
Hub AFW to Center Des Moines: 300 tons
Hub AFW to Center Fairbanks: 38 tons
Hub AFW to Center Fort Wayne: 200 tons
Hub AFW to Center Hartford: 540 tons
Hub AFW to Center Honolulu: 500 tons
Hub AFW to Center Houston: 3,300 tons
Hub AFW to Center Kahului/Maui: 16 tons
Hub AFW to Center Kansas City: 975 tons
Hub AFW to Center Kona: 63 tons
Hub AFW to Center Lakeland: 185 tons
Hub AFW to Center Manchester: 100 tons
Hub AFW to Center Miami: 3,400 tons
Hub AFW to Center Minneapolis: 1,700 tons
Hub AFW to Center Nashville: 650 tons
Hub AFW to Center New Orleans: 550 tons
Hub AFW to Center

**Output Analysis**

The optimization model successfully produced an optimal solution that minimizes total transportation costs while satisfying all operational constraints. The analysis below demonstrates that:

1. All constraints are satisfied (hub capacities, focus city capacities, flow balance, and demand requirements)
2. The solution properly incorporates all decision variables, constraints, and the objective function
3. The output matches expected behavior for a cost-minimizing transportation network

The solver achieved an optimal status with a minimum total cost of **$199,476.25**, representing the most efficient allocation of cargo shipments across Amazon Air's three-tier logistics network.

### B1. Satisfying Optimization Problem Constraints

> Demonstrate that the constraints of the optimization problem are satisfied.

In [22]:
# --- Constraint Verification ---
# 1. Verify Hub Capacity Constraints
print("\n1. Hub Capacity Constraints")
print("-"*80)
for h in hubs:
    outbound_to_focus = sum([x[(h, fc)].varValue for fc in focus_cities if (h, fc) in routes_x])
    outbound_to_centers = sum([y[(h, c)].varValue for c in centers if (h, c) in routes_y])
    total_outbound = outbound_to_focus + outbound_to_centers
    capacity = hub_capacities[h]
    utilization = (total_outbound / capacity) * 100

    print(f"{h}:")
    print(f"  Total Outbound: {total_outbound:,.0f} tons")
    print(f"  Capacity: {capacity:,.0f} tons")
    print(f"  Utilization: {utilization:.2f}%")
    print(f"  Satisfied: {'YES' if total_outbound <= capacity else 'NO'}")
    print()

# 2. Verify Focus City Capacity Constraints
print("\n2. Focus City Capacity Constraints (Inbound)")
print("-"*80)
for fc in focus_cities:
    inbound = sum([x[(h, fc)].varValue for h in hubs if (h, fc) in routes_x])
    capacity = focus_city_capacities[fc]
    utilization = (inbound / capacity) * 100 if capacity > 0 else 0

    print(f"{fc}:")
    print(f"  Total Inbound: {inbound:,.0f} tons")
    print(f"  Capacity: {capacity:,.0f} tons")
    print(f"  Utilization: {utilization:.2f}%")
    print(f"  Satisfied: {'YES' if inbound <= capacity else 'NO'}")
    print()

# 3. Verify Flow Balance at Focus Cities
print("\n3. Flow Balance at Focus Cities")
print("-"*80)
for fc in focus_cities:
    inbound = sum([x[(h, fc)].varValue for h in hubs if (h, fc) in routes_x])
    outbound = sum([z[(fc, c)].varValue for c in centers if (fc, c) in routes_z])
    balance = abs(inbound - outbound)

    print(f"{fc}:")
    print(f"  Inbound: {inbound:,.0f} tons")
    print(f"  Outbound: {outbound:,.0f} tons")
    print(f"  Balance Difference: {balance:.6f} tons")
    print(f"  Satisfied: {'YES' if balance < 0.01 else 'NO'}")
    print()

# 4. Verify Fulfillment Center Demand Constraints
print("\n4. Fulfillment Center Demand Satisfaction")
print("-"*80)
demand_satisfied_count = 0
for c in centers:
    from_hubs = sum([y[(h, c)].varValue for h in hubs if (h, c) in routes_y])
    from_focus = sum([z[(fc, c)].varValue for fc in focus_cities if (fc, c) in routes_z])
    total_received = from_hubs + from_focus
    demand = center_demands[c]
    difference = abs(total_received - demand)

    if difference < 0.01:
        demand_satisfied_count += 1

print(f"Centers with Demand Satisfied: {demand_satisfied_count} / {len(centers)}")
print(f"Satisfaction Rate: {(demand_satisfied_count/len(centers))*100:.1f}%")

# Show a few examples
print("\nSample Verification (first 5 centers):")
for i, c in enumerate(centers[:5]):
    from_hubs = sum([y[(h, c)].varValue for h in hubs if (h, c) in routes_y])
    from_focus = sum([z[(fc, c)].varValue for fc in focus_cities if (fc, c) in routes_z])
    total_received = from_hubs + from_focus
    demand = center_demands[c]

    print(f"  {c}: Received {total_received:,.0f} tons, Demand {demand:,.0f} tons - {'Yes' if abs(total_received - demand) < 0.01 else 'No'}")

# 5. Verify Non-negativity Constraints
print("\n5. Non-negativity Constraints")
print("-"*80)
negative_count = 0
total_variables = len(routes_x) + len(routes_y) + len(routes_z)

for route in routes_x:
    if x[route].varValue < 0:
        negative_count += 1

for route in routes_y:
    if y[route].varValue < 0:
        negative_count += 1

for route in routes_z:
    if z[route].varValue < 0:
        negative_count += 1

print(f"Total Decision Variables: {total_variables}")
print(f"Variables with Negative Values: {negative_count}")
print(f"Non-negativity Satisfied: {'YES - All variables are non-negative' if negative_count == 0 else 'NO'}")


1. Hub Capacity Constraints
--------------------------------------------------------------------------------
CVG:
  Total Outbound: 95,650 tons
  Capacity: 95,650 tons
  Utilization: 100.00%
  Satisfied: YES

AFW:
  Total Outbound: 38,097 tons
  Capacity: 44,350 tons
  Utilization: 85.90%
  Satisfied: YES


2. Focus City Capacity Constraints (Inbound)
--------------------------------------------------------------------------------
Leipzig:
  Total Inbound: 43,470 tons
  Capacity: 85,000 tons
  Utilization: 51.14%
  Satisfied: YES

Hyderabad:
  Total Inbound: 0 tons
  Capacity: 19,000 tons
  Utilization: 0.00%
  Satisfied: YES

San Bernardino:
  Total Inbound: 0 tons
  Capacity: 36,000 tons
  Utilization: 0.00%
  Satisfied: YES


3. Flow Balance at Focus Cities
--------------------------------------------------------------------------------
Leipzig:
  Inbound: 43,470 tons
  Outbound: 43,470 tons
  Balance Difference: 0.000000 tons
  Satisfied: YES

Hyderabad:
  Inbound: 0 tons
  Outbou

The verification above demonstrates that **all five constraint types are satisfied**:

1. **Hub Capacity Constraints**: Both CVG and AFW operate within their capacity limits
2. **Focus City Capacity Constraints**: All focus cities receive shipments within their inbound capacity limits
3. **Flow Balance**: Each focus city's inbound shipments exactly equal outbound shipments (conservation of flow)
4. **Demand Satisfaction**: All 65 fulfillment centers receive exactly their required demand
5. **Non-negativity**: All decision variables have non-negative values

This confirms the solution is feasible and valid.

### B2. Decision Variables, Constraints, and Objective Function

> Demonstrate that the solution includes decision variables, constraints, and the objective function.

The solution properly incorporates all three required components of a linear programming model:

**Decision Variables:**

The model includes three sets of continuous decision variables representing shipment quantities:

- **x_{ij}**: Shipments from hub *i* to focus city *j*
  - Example routes: CVG to Leipzig (43,470 tons)
  - Total variables: 3 valid routes (some hub-focus combinations disallowed)

- **y_{ik}**: Direct shipments from hub *i* to fulfillment center *k*
  - Example routes: CVG to New York (11,200 tons), AFW to Houston (3,300 tons)
  - Total variables: 126 valid routes (65 centers × 2 hubs, minus disallowed routes)

- **z_{jk}**: Shipments from focus city *j* to fulfillment center *k*
  - Example routes: Leipzig to Delhi (19,000 tons), Leipzig → Mumbai (14,800 tons)
  - Total variables: 60 valid routes (some focus-center combinations disallowed)

All decision variables are continuous (allowing fractional tons) and non-negative, as required for transportation quantities.

**Constraints:**

The model enforces five types of constraints to ensure operational feasibility:

1. **Hub Capacity Constraints** (2 constraints): Total outbound shipments from each hub cannot exceed capacity
2. **Focus City Capacity Constraints** (3 constraints): Total inbound shipments to each focus city cannot exceed capacity
3. **Flow Balance Constraints** (3 constraints): Inbound and outbound flow must balance at each focus city
4. **Demand Constraints** (65 constraints): Each fulfillment center must receive exactly its required demand
5. **Non-negativity Constraints** (189 constraints): All shipment quantities must be ≥ 0

Total: **73 explicit constraints** plus non-negativity bounds

**Objective Function:**

The objective function minimizes total transportation cost by summing the cost of all shipments:

**Minimize:** Σ(cost * quantity) across all three route types
```
Total Cost = Σ c_ij * x_ij + Σ c_ik * y_ik + Σ c_jk * z_jk
```

The optimal solution achieves a minimum cost of **$199,476.25**, representing the most cost-effective distribution plan that satisfies all constraints.

### B3. Expected Output Solution

> Explain why the solution that your code outputs matches your expected output.

The solution output matches expected behavior for several key reasons:

**Cost-Minimizing Route Selection**

The solver prioritized lower-cost routes wherever possible:
* Leipzig handles all European/Indian shipments through hub-to-focus city routing rather than expensive direct routes.
* San Bernardino is not utilized, likely because direct hub-to-fulfillment center routes are more cost-effective than adding a focus city intermediary.
* Direct routes from AFW to Texas use the lowest available rate.

**Hub Utilization Patterns**

Hub usage aligns with geographic efficiency:

* CVG handles most European destinations and long-distance domestic routes.
* AFW focuses on western and southern U.S. routes
* Capacity slack at AFW provides operational flexibility.

**Focus City Behavior**

Only Leipzig is used, which is expected because:

* High-demand Indian markets are only accessible through Leipzig to India routing.
* Hyderabad is not used because Indian fulfillment centers (Delhi, Mumbai, Bangalore, Coimbatore) are served directly from Leipzig, bypassing the Hyderabad focus city entirely.
* San Bernardino is bypassed because direct hub-to-center routes within the U.S. are cheaper than adding an intermediary focus city.
* Leipzig serves as the sole international gateway, consolidating all non-U.S. shipments

**Mathematical Optimality**

The "Optimal" solver status confirms this is the best solution:

* The simplex algorithm explored the feasible region and confirmed no lower-cost solution exists.
* All 73 constraints are satisfied with exact precision (demand equations are balanced, flow balance was achieved at all focus cities)
* The solution demonstrates efficient resource utilization: CVG at maximum capacity, AFW with operational buffer, and Leipzig at more moderate utilization with room for operational growth.

**Real-World Reasonableness**

The solution exhibits practical logistics patterns:

* Geographic specialization:
  * CVG handles all international shipments plus eastern U.S. centers.
  * AFW handles western and southern U.S. centers.
* Capacity optimization: CVG is fully utilized, suggesting it's the most cost-effective hub and a potential expansion target.
* Focus city efficiency: Only 1 of 3 focus cities is needed, with Hyderabad and San Bernardino offering no cost advantage over direct routing.
* Two-tier routing for international shipments, direct routing for domestic shipments: Leipzig enables cost-effective European/Indian distribution while U.S. centers are served directly.

**Final Insights**

The model reveals that direct hub-to-center routing is optimal for domestic U.S. shipments, while hub to Leipzig to center routing is optimal for international destinations. Neither Hyderabad or San Bernardino provide sufficient cost savings to justify their use as intermediaty shipment locations, given the available direct routing options at competitive rates.

This output represents the mathematically optimal and operationally practical solution to Amazon Air's distribution problem, achieving a minimum cost of **$199,476.25** while satisfying all demand and capacity constraints.

## C. Reflection

> Provide a reflection on how the development of your approach matched your expectations of the process.

The development of this optimization solution closely aligned with my initial expectations in terms of the systematic methodology, but revealed several surprises regarding problem complexity and solver behavior that deepened my understanding of linear programming applications.

Going into this project, I anticipated that the primary challenge would be correctly translating the mathematical model from Task 2 into executable code. I expected the PuLP library to handle the optimization mechanics automatically once the problem was properly formulated. My assumption was that if the mathematical representation was correct, the coding process would be relatively straightforward—define variables, add constraints, specify the objective function, and solve. I also expected the solution to utilize all three focus cities in some capacity, assuming that Amazon would have designed their network with all facilities serving active roles. The concept of a three-tier hierarchical network suggested balanced utilization across all tiers.

The translation from mathematical notation to Python code proved more nuanced than anticipated. While PuLP's syntax is intuitive, several aspects required careful consideration. Creating the route tuples and filtering for valid combinations by excluding "N/A" entries required more attention than expected. I had to ensure that decision variables were only created for allowable routes, which meant carefully constructing the `routes_x`, `routes_y`, and `routes_z` lists by checking against the cost dictionary. Writing the constraints involved understanding PuLP's `lpSum()` function and how to properly iterate through filtered route combinations. The flow balance constraint at focus cities was particularly interesting, as ensuring that inbound exactly equals outbound required precise indexing of the decision variable dictionaries. Initially, I wasn't certain the model was correct until I saw the "Optimal" status, and the verification code in Section B1 became crucial for building confidence that the solution was not just mathematically optimal, but also operationally valid.

Several outcomes challenged my initial assumptions. The most surprising result was that only Leipzig is used, while Hyderabad and San Bernardino remain completely idle at 0% utilization. This revealed that hierarchical networks don't necessarily require all tiers to be active—sometimes direct routing is more cost-effective than adding intermediary stops. This has real-world implications for Amazon's infrastructure planning. Additionally, CVG operating at exactly 100% capacity while AFW has 14% spare capacity suggests that the network design might benefit from capacity rebalancing or that CVG handles fundamentally different, higher-demand routes. This wasn't immediately obvious from the mathematical formulation alone.

The solver's performance also exceeded my expectations. Despite having 191 decision variables and 73 constraints, it found the optimal solution almost instantly. This demonstrated the power of the simplex algorithm for transportation problems and reinforced the efficiency of modern optimization solvers. Furthermore, the output patterns revealed clear geographic routing logic that made business sense. All European and Indian shipments route through Leipzig, while all U.S. shipments go direct from hubs to centers. The mathematical optimization naturally discovered practical operational strategies without any explicit geographic rules being programmed into the model.

This project reinforced several key lessons about optimization modeling. The quality of the code output directly reflected the quality of the mathematical model from Task 2. Time spent carefully defining constraints and the objective function in mathematical notation paid off during implementation. Writing the constraint verification code was not just about meeting assignment requirements—it was crucial for validating the solution. In a real-world setting, I would never trust optimization output without thorough verification. The solver's decision to leave two focus cities unused wasn't something I would have predicted through intuition alone, which highlights the value of mathematical optimization over intuitive human decision-making.

PuLP proved to be an excellent choice for this problem. Its readable syntax and robust CBC solver handled the problem efficiently. However, I also learned that understanding the underlying linear programming theory is essential—the tool is only as good as the formulation you provide. The systematic workflow of formulate, code, solve, and verify matched my expectations, but the specific solution and the ease of implementation once properly structured exceeded them.

Overall, the development process was mostly aligned with my initial expectations and had only a few surprising discoveries. The process reinforced that optimization is both an art, in terms of formulating the problem correctly, and a science, in terms of applying proven algorithms. While the mathematics and coding were important, the most valuable skill developed was the ability to interpret optimization results in the context of real-world operations and use verification techniques to build confidence in the solution.

## D. Sources

> Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.



* [PuLP Documentation - A Transportation Problem](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html)
* WGU Course Materials